# Apollo Analytical Model Explorer

Build and simulate the application of Apollo models against an exhaustive dataset.

In [4]:

def project_model_over_trace(data):
    with trace_detail:
        # compile a model
        print("Constructing models.")
        all_models = construct_model_from_flush(data, 'apollo.flush')

        print("Processing trace.")

        trace_pos = 0
        trace_pos_total = len(data['apollo.trace'].index)
        trace_progress.max = 1000

        all_times = []
                
        dfb = data['policy.times.best']
        dfd = data['policy.times.default']
        best_col_time    = dfb.columns.get_loc('min_time')
        best_col_policy  = dfb.columns.get_loc('policy_index')
        default_col_time = dfd.columns.get_loc('min_time')
        
        print("trace_pos,step,region_name,policy_index,model_policy,best_policy," \
              + "num_elements,time_exec,default_time,best_time")
        
        for row in data['apollo.trace'].itertuples():
            step          = int(row.step)
            region_name   = str(row.region_name)
            policy_index  = int(row.policy_index)
            num_threads   = int(row.num_threads)
            num_elements  = int(row.num_elements)
            time_exec     = float(row.time_exec)

            if (trace_pos > 1000):
                break
            
            #TODO: Lookup table for model_time
            model_policy = int(all_models[region_name].predict([[num_elements]]))
            model_time   = 0.0
            
            
            best_row = dfb[(dfb.step==(step + 1))
                         & (dfb.region_name==region_name)
                         & (dfb.num_elements==num_elements)].first_valid_index()
            
            default_row = dfd[(dfd.step==(step + 1))
                            & (dfd.region_name==region_name)
                            & (dfd.num_elements==num_elements)].first_valid_index()
            
            
            best_time     = dfb.iat[best_row,    best_col_time]
            best_policy   = dfb.iat[best_row,    best_col_policy]
            default_time  = dfd.iat[default_row, default_col_time]
            
            all_times.append((trace_pos, step, time_exec, best_time, default_time))
            
            print("%d,%d,%s,%d,%d,%d,%d,%1.8f,%1.8f,%1.8f" % (
                trace_pos,
                step,
                region_name,
                policy_index,
                model_policy,
                best_policy,
                num_elements,
                time_exec,
                default_time,
                best_time))

          
            trace_pos += 1
            trace_progress.value = trace_pos
        #
        trace_progress.value = trace_progress.max
        trace_accordion.selected_index = 1
        
        dft = pd.DataFrame(all_times,
                           columns=('trace_pos', 'step', 'time_exec', 'time_best', 'time_default'))
        with trace_summary:
            fig,ax = plt.subplots()
            fig.set_size_inches(8, 3, forward=True)
            dft.plot(x='trace_pos', y='time_exec', ax=ax, linestyle='-', title="Cleverleaf Trace and Model Analysis")
            dft.plot(x='trace_pos', y='time_best', ax=ax, linestyle='--')
            dft.plot(x='trace_pos', y='time_default', ax=ax, linestyle=':')
            ax.legend(["trace", "best", "default"], loc='upper right')
            ax.tick_params(axis='y', which='minor', left=True)
            ax.tick_params(axis='x', which='minor', bottom=True)
            plt.grid(True)
            plt.xlabel('Cleverleaf Trace Progress')
            plt.ylabel('Time (seconds)')
            plt.show()
    return


#----------

%run './common_routines.ipynb'
%matplotlib notebook

data = {}
data['path'] = '/g/g17/wood67/src/apollo/data/intel/001.node.001.rank'
data['apollo.tracefile'] = 'trace.policy.0.csv'
data['apollo.flushfile'] = 'flush.grouped.csv'
data['apollo.stepsfile'] = 'steps.apollo.silent.csv'
data['normal.stepsfile'] = 'steps.normal.silent.csv'
data['policy.times.bestfile']  = 'policy.times.best.csv'
data['policy.times.defaultfile'] = 'policy.times.default.csv'

data = load_csv_data(data)

#trace_thread = threading.Thread(
#    target=project_model_over_trace,
#    args=(data))
#trace_thread.start()

project_model_over_trace(data)

#plot_apollo_vs_normal(data)

print("\nDone.")
return





Done.
